<a href="https://colab.research.google.com/github/PTettey/Recommender-FastAi/blob/master/Automotive_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 8.7MB/s 
     |████████████████████████████████| 194kB 16.7MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 1.2MB 35.2MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 12.8MB 249kB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *

## A Look at the Data

The data used for this exercise was from the Amazon Product Data. The original datasets contains product reviews and metadata from Amazon.

This includes reviews in the form of ratings, text and helpfulness votes. It also has product metadata (descriptions, category information, price, brand, and image features), and links.

We however choose the Automotive Ratings dataset from the "small" Subset Experimentation files.




In [3]:
from fastai.collab import *
from fastai.tabular.all import *
path = Path('/content/gdrive/MyDrive/fastai-v3/data/Automotive')

In [4]:
path.ls()

(#1) [Path('/content/gdrive/MyDrive/fastai-v3/data/Automotive/Automotive.csv')]

Unlike the [MovieLens](https://grouplens.org/datasets/movielens/) dataset used in the __FastAi__ lesson, this Subset dataset contians 7990166 rows and 4 columns. The made up of `Item`,`User`,`Rating` and `Timestamp` respectively.

`Item` provided the product IDs. The IDs are alphanumeric.

`User` column gave the IDs for the users. Just like the Items, the User IDs are also alphanumeric.

`Rating` column contained the ratings given by the users. The ratings ranges from 1.0 to 5.0
 
 and the `Timestamp` gave the time at which said ratings took place.

In [5]:
data = pd.read_csv("/content/gdrive/MyDrive/fastai-v3/data/Automotive/Automotive.csv", header=None,
                      names=['Item','User','Rating','Timestamp'])
data.head(10)

,Item,User,Rating,Timestamp
0,0209688726,A3ELV1VUCZZGL4,4.0,1439337600
1,0209688726,A2FD3FZ2WV6KHQ,2.0,1439337600
2,0209688726,A39NRKABAVSVSE,2.0,1439251200
3,0209688726,AQSBL5KODHSGW,5.0,1438128000
4,0209688726,AU45OQPK2J5JF,5.0,1436572800
5,0209688726,A5TACMBV2GTBS,5.0,1433980800
6,0209688726,A8WEXFRWX1ZHH,4.0,1430438400
7,0209688726,A262RPUOSCPP51,5.0,1524182400
8,0209688726,ABRUILETFGA81,5.0,1524096000
9,0209688726,ABCA1A8E4DGV1,1.0,1524096000


In [6]:
len(data)

7990166

We tried using the total data for this. It however took forever to run, so we decided to resize it down and use a much smaller sample size.

We reduced the number of rows to 800000. We also reordered the columns and placed the `User` column before the `Item` column.

In [7]:
Sample_data = data.sample(n = 800000)

In [8]:
len(Sample_data)

800000

In [9]:
Sample_data_reorder = Sample_data[['User','Item','Rating','Timestamp']] 

In [10]:
Sample_data_reorder.head()

,User,Item,Rating,Timestamp
2555848,A1N0VCS41TGZF0,B007ECFURK,5.0,1440201600
427995,A28HU2E2Y0OTLJ,B000BYB2K2,5.0,1396224000
4967238,APV7YI6K3PL8M,B00062YNBE,5.0,1372032000
6717806,A3TTAJQUAX117Q,B00F3I14WC,5.0,1471219200
1201487,ANIZGD7L3LULC,B000W0KTMW,1.0,1369094400


# Recommend with Collaborative Filtering



## Creating the Dataloaders

We can then build a `DataLoaders` object from this table. By default, it takes the first column for the __User__, the second column for the __Item__, and the third column for the __Rating__. 

We decided to use a batch size of 64.

In [13]:
dls = CollabDataLoaders.from_df(Sample_data_reorder, bs = 64)
dls.show_batch()

,User,Item,Rating
0,AEHGZNP1BR478,B00MQ1D7M4,5.0
1,ANFWEHA8U4LJB,B00QXXVPF2,5.0
2,A1QR6C8ISNV99N,B00FSU1SQM,5.0
3,A2UBBI4DWVAWP5,B014FFQJEG,5.0
4,A19DR0R9U9R2HS,B004YGRL62,5.0
5,A2490DCYUQ359U,B00F3W06CM,5.0
6,A3N1I9CJIDPGJ0,B009YYVSZO,3.0
7,A1HUEPDNAZF8S1,B009XOTRBM,5.0
8,A7JNL4W2OGCDZ,B000CPJMCU,2.0
9,A1Z8BN4PPX6H9Z,B01EJ08A44,3.0


In [24]:
y_range = [0.5,5.5]

In [25]:
learn = collab_learner(dls, n_factors=50, y_range=y_range, wd=1e-1)

To get better results, we tried using the Y range. This is what makes the final activation a sigmoid function. This helps make prediction within the specified range.

To use sigmoid function and stay withing the range, we extend and/or reduce the Y range. The lowest rating for this dataset set is 1 and the highest is 5. We therefore choose a range between 0.5 to 5.5.

We also add a Weight decay of `1e-1` to help avoid overfitting.

In [26]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,2.939244,2.947483,04:26
1,2.919961,2.924299,04:25
2,2.918597,2.920055,04:25
3,2.867791,2.916124,04:26
4,2.761037,2.915778,04:26


In [27]:
learn.save('Automotive-ratings')

Path('models/Automotive-ratings.pth')

In [28]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(665054, 50)
  (i_weight): Embedding(266524, 50)
  (u_bias): Embedding(665054, 1)
  (i_bias): Embedding(266524, 1)
)

## Intepreting Bias

The Embeddings should contain some interesting feature which we can extract to have a better insight.

We will extract the biases as well as their mean ratings in order to get informations about Automotives that are generally rated low or high no matter what user is rating them.

In [29]:
Bias = learn.model.i_bias.weight.squeeze()
idxs = Bias.argsort(descending=True)[:5]
[dls.classes['Item'][i] for i in idxs]

['B014TZAUU6', 'B00068XCQU', 'B000KPU8F2', 'B000CITK8S', 'B0015Y5848']

In [30]:
Bias = learn.model.i_bias.weight.squeeze()
idxs = Bias.argsort(descending=False)[:5]
[dls.classes['Item'][i] for i in idxs]

['B003NU0EM8', 'B00ZVJ32QE', 'B0015D88U0', 'B01GCB730U', 'B00BCRMBGQ']

So the top thousand are the automotive that have been rated the most, and so there hopefully automotive that we might have seen.That's the only reason I'm doing this. So I've called this `top_automotive` by which I mean not good automotive, just automotive we likely to have seen.

Not surprisingly, Star Wars is the one, at that point, the most people had put a rating to. Independence Day, there you go. We can then take our learner that we trained and asked it for the bias of the items listed here.